# Will Gasser | 2/20/2025 | MNIST NN

This is a simple NN with back propogation to formalize my understanding of ML and NN concepts.

I am limiting myself to having imports: numpy, random, time (just for stats), pandas (just for data read), and matplotlib for visualization after.

Let us see how fast I can complete this, oh and no LLMs for any functions, I will note if I use it for syntax.

Init commit now: 2/20/2025 @ 9:14 EST

Finish: TBD

In [2]:
import numpy as np
import random
import time
import pandas as pd
import matplotlib.pyplot as plt

In [9]:
TRAIN_DATA = np.array(pd.read_csv('mnist_train.csv'))
TEST_DATA = np.array(pd.read_csv('mnist_test.csv'))

In [37]:
# DATA FUNCTIONS

BATCH_SIZE = np.shape(TRAIN_DATA)
BATCH_INDICES = list(range(0, BATCH_SIZE[0]))

def shuffle_batch() -> None:
    random.shuffle(BATCH_INDICES)
    
def get_data(index: int) -> np.ndarray:
    return TRAIN_DATA[index]

def get_label(data: np.ndarray) -> int:
    return data[0]
    
def get_features(data: np.ndarray) -> np.ndarray:
    return data[np.newaxis, 1:] / 255.0

In [ ]:
# FORWARD / BACKWARD PROPOGATION FUNCTIONS

def activation_sigmoid(Z: np.ndarray) -> np.ndarray: # Z is activations
    return (1 / (1+ np.exp(-Z)))

def foward_pass(activations: np.ndarray, layers: list, bias: list) -> np.ndarray:
        for i in range(len(layers)): # at each neural layer
            layer = layers[i] # specific layer, specific bias
            B = bias[i]
            activations = activation_sigmoid(np.dot(activations, layer) + B)
        
        print(f'{np.shape(activations)}')
        for output, i in enumerate(activations.flatten()):
            print(f'{output}: {i}')    
            
def back_propogation(activations: np.ndarray, layers: list, bias: list, gradients: list) -> None:
            

In [47]:
'''
###################################################################
### THIS CELL SETS THE MODEL WEIGHTS/BIAS BACK TO RANDOM VALUES ###
###################################################################
'''

# MODEL INSTANTIATION

weights784 = np.random.uniform(low=-1.0, high=1.0, size=(784, 256)) # INPUT LAYER
bias256 = np.zeros((1,256))

weights256 = np.random.uniform(low=-1.0, high=1.0, size=(256, 128)) # Hidden layer 1
bias128 = np.zeros((1,128))

weights128 = np.random.uniform(low=-1.0, high=1.0, size=(128, 64)) # Hidden layer 2
bias64 = np.zeros((1,64))

weights64= np.random.uniform(low=-1.0, high=1.0, size=(64, 10)) # OUTPUT
bias10 = np.zeros((1,10))

layers = [weights784, weights256, weights128, weights64]
bias = [bias256, bias128, bias64, bias10]

In [ ]:
# DATA PRE-PROCESSING
print(f'Data processing, shuffling batch of size {len(BATCH_INDICES)}.')
start = time.time()
shuffle_batch()
shuffle_time = time.time() - start
print(f'Data shuffling complete, took {shuffle_time} seconds.')

test_data = get_data(BATCH_INDICES[0])
test_example = get_features(test_data)
test_label = get_label(test_data)

# TEST FORWARD PROP
print('COMMENCING FORWARD PROPOGATION TEST')

forward_pass(test_example, layers, bias)

print('COMPLETE')

Data processing, shuffling batch of size 59999.
Data shuffling complete, took 0.020573854446411133 seconds.
COMMENCING FORWARD PROPOGATION TEST
(1, 10)
0: 0.07325593185183116
1: 0.06112019921301674
2: 0.7922869461998663
3: 0.015844117248661544
4: 0.8910250537599986
5: 0.013816483197082619
6: 0.09339604657095447
7: 0.199764021945061
8: 0.04933574378299642
9: 0.6168582059440221
COMPLETE
